In [1]:
import numpy as np

In [35]:
# 读取训练集
user_num = 943
item_num = 1682

ratings = np.zeros((user_num + 1, item_num + 1), int)
y_ui = np.zeros((user_num + 1, item_num + 1), int)

with open('../data/ml-100k/u1.base', 'r') as file:
    for line in file.readlines():
        user, item, rating, timestamp = line.split('	')
        ratings[int(user)][int(item)] = int(rating)
        y_ui[int(user)][int(item)] = 1

p = y_ui.sum()
density = p / (user_num * item_num)
r_ = ratings.sum() / p
n = y_ui.sum()

r_1 = ratings.sum(axis=0)
r_2 = ratings.sum(axis=1)
r_3 = y_ui.sum(axis=0)
r_4 = y_ui.sum(axis=1)

In [36]:
# 求四个参数
r_u = np.zeros(user_num + 1, float)
for i in range(1, user_num + 1):
    r_u[i] = (r_ if r_4[i] == 0 else r_2[i] / r_4[i])

r_i = np.zeros(item_num + 1, float)
for i in range(1, item_num + 1):
    r_i[i] = (r_ if r_3[i] == 0 else r_1[i] / r_3[i])

b_u = np.zeros(user_num + 1, float)
for i in range(1, user_num + 1):
    b_u[i] = (0 if r_4[i] == 0 else (y_ui[i] * (ratings[i] - r_i)).sum() / r_4[i])

b_i = np.zeros(item_num + 1, float)
for i in range(1, item_num + 1):
    b_i[i] = (0 if r_3[i] == 0 else (y_ui[:, i] * (ratings[:, i] - r_u)).sum() / r_3[i])

In [54]:
# 损失函数
def MAE(ratings_hat, ratings, y_ui, n):
    return abs(ratings_hat * y_ui - ratings).sum() / n

def RMSE(ratings_hat, ratings, y_ui, n):
    return (((ratings_hat * y_ui - ratings) ** 2).sum() / n) ** 0.5

In [61]:
# 预测
ratings_test = np.zeros((user_num + 1, item_num + 1), int)
y_ui_test = np.zeros((user_num + 1, item_num + 1), int)

with open('../data/ml-100k/u1.test', 'r') as file:
    for line in file.readlines():
        user, item, rating, timestamp = line.split('	')
        ratings_test[int(user)][int(item)] = int(rating)
        y_ui_test[int(user)][int(item)] = 1

test_num = y_ui_test.sum()

# 评分规则
ratings_hat1 = np.zeros_like(ratings_test, float)
ratings_hat2 = np.zeros_like(ratings_test, float)
ratings_hat3 = np.zeros_like(ratings_test, float)
ratings_hat4 = np.zeros_like(ratings_test, float)
ratings_hat5 = np.zeros_like(ratings_test, float)
ratings_hat6 = np.zeros_like(ratings_test, float)

for i in range(1, user_num + 1):
    ratings_hat1[i, :] = r_u[i]

for i in range(1, item_num + 1):
    ratings_hat2[:, i] = r_i[i]

for i in range(1, user_num + 1):
    for j in range(1, item_num + 1):
        ratings_hat3[i, j] = r_u[i] / 2 + r_i[j] / 2

for i in range(1, user_num + 1):
    for j in range(1, item_num + 1):
        ratings_hat4[i, j] = b_u[i] + r_i[j]

for i in range(1, user_num + 1):
    for j in range(1, item_num + 1):
        ratings_hat5[i, j] = r_u[i] + b_i[j]

for i in range(1, user_num + 1):
    for j in range(1, item_num + 1):
        ratings_hat6[i, j] = r_ + b_u[i] + b_i[j]

print(f"{RMSE(ratings_hat1, ratings_test, y_ui_test, test_num):.4f}, {MAE(ratings_hat1, ratings_test, y_ui_test, test_num):.4f}")
print(f"{RMSE(ratings_hat2, ratings_test, y_ui_test, test_num):.4f}, {MAE(ratings_hat2, ratings_test, y_ui_test, test_num):.4f}")
print(f"{RMSE(ratings_hat3, ratings_test, y_ui_test, test_num):.4f}, {MAE(ratings_hat3, ratings_test, y_ui_test, test_num):.4f}")
print(f"{RMSE(ratings_hat4, ratings_test, y_ui_test, test_num):.4f}, {MAE(ratings_hat4, ratings_test, y_ui_test, test_num):.4f}")
print(f"{RMSE(ratings_hat5, ratings_test, y_ui_test, test_num):.4f}, {MAE(ratings_hat5, ratings_test, y_ui_test, test_num):.4f}")
print(f"{RMSE(ratings_hat6, ratings_test, y_ui_test, test_num):.4f}, {MAE(ratings_hat6, ratings_test, y_ui_test, test_num):.4f}")

1.0630, 0.8502
1.0334, 0.8276
0.9985, 0.8085
0.9602, 0.7574
0.9758, 0.7696
0.9623, 0.7613
